## Initial Data Collection and Cleaning
In this notebook we will:
1. Use two different datasets from Kaggle that took game information from steam.com 
2. We will do slight cleaning on each dataset then merge them to make one
3. Further cleaning will be done on the combined dataset 
4. This dataset will then be used for out recommendation system

***
## Import neccessary packages:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from collections import Counter 
from itertools import dropwhile
import matplotlib.pyplot as plt
import pickle

*** 
## Data Collecting and Cleaning:
This is where we will collect and clean the datasets

In [2]:
steam = pd.read_csv('data/steam.csv')

steam_games = pd.read_csv('data/steam_games.csv')

### First dataset
Start by looking at the first data set we called in and understand what information we have.

In [3]:
steam.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [4]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
appid               27075 non-null int64
name                27075 non-null object
release_date        27075 non-null object
english             27075 non-null int64
developer           27075 non-null object
publisher           27075 non-null object
platforms           27075 non-null object
required_age        27075 non-null int64
categories          27075 non-null object
genres              27075 non-null object
steamspy_tags       27075 non-null object
achievements        27075 non-null int64
positive_ratings    27075 non-null int64
negative_ratings    27075 non-null int64
average_playtime    27075 non-null int64
median_playtime     27075 non-null int64
owners              27075 non-null object
price               27075 non-null float64
dtypes: float64(1), int64(8), object(9)
memory usage: 3.7+ MB


#### steam dataframe descriptions 
|column name | description |
| --- | ---|
|appid| Unique identifier for each title|
|name| Title of app (game)|
|release_date | Release date in format YYYY-MM-DD|
|english | Language support: 1 if is in English| 
|developer| Name (or names) of developer(s). Semicolon delimited if multiple|
|publisher | Name (or names) of publisher(s). Semicolon delimited if multiple |
|platforms | Semicolon delimited list of supported platforms. At most includes: windows;mac;linux| 
|required_age | Minimum required age according to PEGI UK standards. Many with 0 are unrated or unsupplied|
|categories| Semicolon delimited list of game categories, e.g. single-player; multi-player|
|genres|Semicolon delimited list of game genres, e.g. action;adventure|
|steamspy_tags| Semicolon delimited list of top steamspy game tags, similar to genres but community voted, e.g. action;adventure|
|achievements|Number of in-games achievements, if any|
|positive_ratings|Number of positive ratings, from SteamSpy|
|negative_ratings|Number of negative ratings, from SteamSpy|
|average_playtime|Average user playtime, from SteamSpy|
|median_playtime|Median user playtime, from SteamSpy|
|owners|Estimated number of owners. Contains lower and upper bound (like 20000-50000)|
|price|Current full price of title in GBP, (pounds sterling)|


### Second dataset
Next look at the second data set we called in and understand what information we have there.

In [5]:
steam_games.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [6]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 20 columns):
url                         40833 non-null object
types                       40831 non-null object
name                        40817 non-null object
desc_snippet                27612 non-null object
recent_reviews              2706 non-null object
all_reviews                 28470 non-null object
release_date                37654 non-null object
developer                   40490 non-null object
publisher                   35733 non-null object
popular_tags                37888 non-null object
game_details                40313 non-null object
languages                   40797 non-null object
achievements                12194 non-null float64
genre                       40395 non-null object
game_description            37920 non-null object
mature_content              2897 non-null object
minimum_requirements        21069 non-null object
recommended_requirements    21075 non-null

#### steam_games dataframe descriptions 
|column name | description |
| --- | ---|
|url| Url of a game|
|types|type of package - app, sub or bundle|
|name|Name of a game|
|desc_snippet|short description of a game|
|recent_reviews|recent reviews|
|all_reviews|all reviews|
|release_date|release date|
|developer|developer of a game|
|publisher|publisher or publishers of a game|
|popular_tags|tags|
|game_details|details of a game|
|languages|supported languages|
|achievements|number of achievements|
|genre|genre(s) of a game|
|game_description|game description|
|mature_content|description of mature content in a game|
|minimum_requirements|minimum specs for a game|
|recommended_requirements|recommended specs for a game|
|original_price|price without discount|
|discount_price|price with discount|


### Merge and Clean
Now we will merge the two data sets by the game name and do further cleaning.

In [7]:
steam['name'] = steam['name'].str.replace(r'[^\w\s]+', '').str.strip().str.lower()
steam_games['name'] = steam_games['name'].str.replace(r'[^\w\s]+', '').str.strip().str.lower()

df = steam.merge(steam_games, left_on='name', right_on='name')

In [8]:
df.head()

,appid,name,release_date_x,english,developer_x,publisher_x,platforms,required_age,categories,genres,...,game_details,languages,achievements_y,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,10,counterstrike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,"Multi-player,Online Multi-Player,Local Multi-P...","English,French,German,Italian,Spanish - Spain,...",NaN,Action,About This Game Play the world's number 1 onl...,Mature Content Description The developers de...,NaN,NaN,$9.99,$65.70
1,20,team fortress classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,"Multi-player,Online Multi-Player,Local Multi-P...","English,French,German,Italian,Spanish - Spain,...",NaN,Action,About This Game One of the most popular onlin...,Mature Content Description The developers de...,NaN,NaN,$4.99,$65.70
2,30,day of defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,"Multi-player,Valve Anti-Cheat enabled","English,French,German,Italian,Spanish - Spain",NaN,Action,About This Game Enlist in an intense brand of...,NaN,NaN,NaN,$4.99,$65.70
3,40,deathmatch classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,"Multi-player,Online Multi-Player,Local Multi-P...","English,French,German,Italian,Spanish - Spain,...",NaN,Action,About This Game Enjoy fast-paced multiplayer ...,NaN,NaN,NaN,$4.99,$65.70
4,50,halflife opposing force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,"Single-player,Multi-player,Valve Anti-Cheat en...","English,French,German,Korean",NaN,Action,About This Game Return to the Black Mesa Rese...,NaN,NaN,NaN,$4.99,$65.70


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19747 entries, 0 to 19746
Data columns (total 37 columns):
appid                       19747 non-null int64
name                        19747 non-null object
release_date_x              19747 non-null object
english                     19747 non-null int64
developer_x                 19747 non-null object
publisher_x                 19747 non-null object
platforms                   19747 non-null object
required_age                19747 non-null int64
categories                  19747 non-null object
genres                      19747 non-null object
steamspy_tags               19747 non-null object
achievements_x              19747 non-null int64
positive_ratings            19747 non-null int64
negative_ratings            19747 non-null int64
average_playtime            19747 non-null int64
median_playtime             19747 non-null int64
owners                      19747 non-null object
price                       19747 non-null float6

#### To drop from df: 

|column name | info | why |
| --- | --- | --- |
|`appid`|`19744 non-null int64`|no need|
|`english`|`19744 non-null int64`|no need|
|`url`|`19744 non-null object`|no need|
|`recent_reviews`|`2465 non-null object`|not enough|
|`release_date_y`|`19734 non-null object`|doubled with `release_date_x`|
|`developer_y`|`19739 non-null object`|doubled with `developer_x`|
|`publisher_y`|`19733 non-null object`|doubled with `publisher_x`|
|`game_details`|`19739 non-null object`|same as `categories`|
|`achievements_y`|`11347 non-null float64`|doubled with `achievements_x`|
|`genre`|`19737 non-null object`|doubled with `genres`|
|`game_description`|`19734 non-null object`|same as `desc_snippet`|
|`minimum_requirements`|`10370 non-null object`|not enough|
|`recommended_requirements`|`10375 non-null object`|not enough|
|`original_price`|`19659 non-null object`|doubled with `price`|
|`discount_price`|`6458 non-null object`|not enough|

We are also going to drop the columns on reviews and play times as we only care about number of sales which is on the nunber of owners. These columns are:

|column name | info |
| --- | --- |
|`positive_ratings`|`19747 non-null int64`|
|`negative_ratings`|`19747 non-null int64`|
|`all_reviews`|`9120 non-null object`|
|`average_playtime`|`19747 non-null int64`|
|`median_playtime`|`19747 non-null int64`|

In [10]:
to_drop = ['appid', 'english', 'url', 'recent_reviews', 'release_date_y', 'developer_y', 'publisher_y',
           'game_details', 'achievements_y', 'genre', 'game_description','steamspy_tags', 
           'minimum_requirements', 'recommended_requirements', 'original_price', 'discount_price',
          'positive_ratings', 'negative_ratings', 'all_reviews', 'average_playtime', 'median_playtime']

df = df.drop(to_drop, axis = 1) 

df = df.rename(columns={'release_date_x': 'release_date', 'developer_x': 'developer', 'publisher_x' : 'publisher',
                  'achievements_x' : 'achievements'})

In [11]:
df.drop(df[df['price'] == 0].index, inplace = True) 

In [12]:
df = df.fillna(value = {'mature_content': 'none'})

df['owners'] = df['owners'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2).astype(int)

df['est_revenue'] = df['owners'] * df['price']

df['year'] = pd.DatetimeIndex(df['release_date']).year
df['month'] = pd.DatetimeIndex(df['release_date']).month

import calendar
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df['month'] = df['month'].str.lower()

In [13]:
spring = ['mar', 'apr', 'may']
for i in spring:
    df.loc[df['month'].str.contains(i), 'season'] = 'spring'
summer = ['jun', 'jul', 'aug']
for i in summer:
    df.loc[df['month'].str.contains(i), 'season'] = 'summer'
fall = ['sep', 'oct','nov']
for i in fall:
    df.loc[df['month'].str.contains(i), 'season'] = 'fall'
winter = ['jan', 'feb', 'dec']
for i in winter:
    df.loc[df['month'].str.contains(i), 'season'] = 'winter'

In [14]:
df = df.dropna()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17930 entries, 0 to 19746
Data columns (total 20 columns):
name              17930 non-null object
release_date      17930 non-null object
developer         17930 non-null object
publisher         17930 non-null object
platforms         17930 non-null object
required_age      17930 non-null int64
categories        17930 non-null object
genres            17930 non-null object
achievements      17930 non-null int64
owners            17930 non-null int64
price             17930 non-null float64
types             17930 non-null object
desc_snippet      17930 non-null object
popular_tags      17930 non-null object
languages         17930 non-null object
mature_content    17930 non-null object
est_revenue       17930 non-null float64
year              17930 non-null int64
month             17930 non-null object
season            17930 non-null object
dtypes: float64(2), int64(4), object(14)
memory usage: 2.9+ MB


#### df dataframe descriptions after cleaning 
|column name | description |
| --- | ---|
|name| Title of app (game)|
|release_date| Release date in format YYYY-MM-DD|
|developer| Name (or names) of developer(s). Semicolon delimited if multiple|
|publisher| Name (or names) of publisher(s). Semicolon delimited if multiple |
|platforms | Semicolon delimited list of supported platforms. At most includes: windows;mac;linux| 
|required_age | Minimum required age according to PEGI UK standards. Many with 0 are unrated or unsupplied|
|categories| Semicolon delimited list of game categories, e.g. single-player; multi-player|
|genres|Semicolon delimited list of game genres, e.g. action;adventure|
|steamspy_tags| Semicolon delimited list of top steamspy game tags, similar to genres but community voted, e.g. action;adventure|
|achievements|Number of in-games achievements, if any|
|positive_ratings|Number of positive ratings, from SteamSpy|
|negative_ratings|Number of negative ratings, from SteamSpy|
|average_playtime|Average user playtime, from SteamSpy|
|median_playtime|Median user playtime, from SteamSpy|
|owners|Estimated owners|
|price|Current full price of title in GBP, (pounds sterling)|
|types|type of package - app, sub or bundle|
|name|Name of a game|
|desc_snippet|short description of a game|
|all_reviews|all reviews|
|popular_tags|tags|
|languages|supported languages|
|est_revenue| estimated revenue (owners * price)|
|year|year game was released|
|month|month game was released|
|season|season game was released|

In [16]:
df['developer'] = df['developer'].str.strip().str.lower()
df['publisher'] = df['publisher'].str.strip().str.lower()
df['types'] = df['types'].str.strip().str.lower()
df['mature_content'] = df['mature_content'].str.strip().str.lower()

df['platforms'] = df['platforms'].str.strip().str.lower().str.replace(' ','').str.split(';').str.join(' ')
df['genres'] = df['genres'].str.strip().str.lower().str.replace(' ','').str.split(';').str.join(' ')
df['categories'] = df['categories'].str.strip().str.lower().str.replace(' ','').str.split(';').str.join(' ')
df['popular_tags'] = df['popular_tags'].str.strip().str.lower().str.replace(' ','').str.split(',').str.join(' ')
df['languages'] = df['languages'].str.strip().str.lower().str.replace(' ','').str.split(',').str.join(' ')

#### These next two functions are helpful in removing any labels that only show up once as we want to focus on tags that are more common than that. 

In [17]:
def find_elements(column):
    all_elements = df[column].map(str).values.tolist()
    all_elements = ' '.join(all_elements)
    all_elements = all_elements.split()
    print('first elements len', len(all_elements))

    cnt = Counter()
    for word in all_elements:
        cnt[word] += 1
    print('first cnt len', len(cnt))

    for key, count in dropwhile(lambda key_count: key_count[1] >= 2, cnt.most_common()):
        del cnt[key]
    print('second cnt len', len(cnt))

    for i in range(0, len(all_elements)): 
        all_elements[i] = ''.join(all_elements[i]) 

    UniqW = Counter(all_elements) 

    all_elements = ' '.join(cnt.keys())
    all_elements = all_elements.split(' ')
    print('second elements len', len(all_elements))
    return cnt, all_elements

In [18]:
def remove_elements(column, all_elements): 
    df[column] = df[column].str.split(' ')
    for index, row in df.iterrows():
        current_elements = row[column]
        new_elements = []
        for element in current_elements:
            if element in all_elements:
                new_elements.append(element)
        df.at[index, column] = new_elements
    df[column] = df[column].str.join(' ')

In [19]:
lang_cnt, all_lang = find_elements('languages')

first elements len 64263
first cnt len 33
second cnt len 30
second elements len 30


In [20]:
remove_elements('languages', all_lang)

In [21]:
tags_cnt, all_tags = find_elements('popular_tags')

first elements len 150695
first cnt len 376
second cnt len 373
second elements len 373


In [22]:
# also drop earlyaccess, greatsoundtrack, masterpiece, kickstater
all_tags.remove('earlyaccess')
all_tags.remove('greatsoundtrack')
all_tags.remove('masterpiece')
all_tags.remove('kickstarter')

In [23]:
remove_elements('popular_tags', all_tags)

In [24]:
genres_cnt, all_genres = find_elements('genres')

first elements len 49428
first cnt len 27
second cnt len 26
second elements len 26


In [25]:
# lets also drop photoediting and 'massivelymultiplayer'
all_genres.remove('photoediting')
all_genres.remove('massivelymultiplayer')
all_genres.remove('earlyaccess')

In [26]:
remove_elements('genres', all_genres)

In [27]:
categories_cnt, all_categories = find_elements('categories')

first elements len 61367
first cnt len 27
second cnt len 27
second elements len 27


In [28]:
remove_elements('categories', all_categories)

#### Lets put all the text columns into one corpus for each game for a text vectorizer in the recommendation system.

In [29]:
text_cols = ['developer', 'publisher', 'platforms', 'categories', 'types', 'desc_snippet', 'popular_tags',
             'languages', 'mature_content']

df['soup'] = ''

for i in text_cols:
    df['soup'] += (' ' + df[i].map(str))

In [54]:
df.head()

,name,release_date,developer,publisher,platforms,required_age,categories,genres,achievements,owners,...,types,desc_snippet,popular_tags,languages,mature_content,est_revenue,year,month,season,soup
0,counterstrike,2000-11-01,valve,valve,windows mac linux,0,multi-player onlinemulti-player localmulti-pla...,action,0,15000000,...,app,Play the world's number 1 online action game. ...,action fps multiplayer shooter classic team-ba...,english french german italian spanish-spain si...,mature content description the developers des...,107850000.0,2000,nov,fall,valve valve windows mac linux multi-player on...
1,team fortress classic,1999-04-01,valve,valve,windows mac linux,0,multi-player onlinemulti-player localmulti-pla...,action,0,7500000,...,app,One of the most popular online action games of...,action fps multiplayer classic shooter team-ba...,english french german italian spanish-spain ko...,mature content description the developers des...,29925000.0,1999,apr,spring,valve valve windows mac linux multi-player on...
2,day of defeat,2003-05-01,valve,valve,windows mac linux,0,multi-player valveanti-cheatenabled,action,0,7500000,...,app,Enlist in an intense brand of Axis vs. Allied ...,fps worldwarii multiplayer shooter action war ...,english french german italian spanish-spain,none,29925000.0,2003,may,spring,valve valve windows mac linux multi-player va...
3,deathmatch classic,2001-06-01,valve,valve,windows mac linux,0,multi-player onlinemulti-player localmulti-pla...,action,0,7500000,...,app,Enjoy fast-paced multiplayer gaming with Death...,action fps classic multiplayer shooter first-p...,english french german italian spanish-spain ko...,none,29925000.0,2001,jun,summer,valve valve windows mac linux multi-player on...
4,halflife opposing force,1999-11-01,gearbox software,valve,windows mac linux,0,single-player multi-player valveanti-cheatenabled,action,0,7500000,...,app,Return to the Black Mesa Research Facility as ...,fps action classic sci-fi singleplayer shooter...,english french german korean,none,29925000.0,1999,nov,fall,gearbox software valve windows mac linux sing...


#### Pickle data to be used in other notebooks
- whole_data to be used for the visulizations 
- simple_df to be used for the recommendations

In [53]:
df.to_pickle('data/whole_data.pkl')

In [32]:
simple_df  = df[['name', 'owners', 'price', 'est_revenue', 'season', 'year', 'soup']].copy()
simple_df.to_pickle('rec_app/simple.pkl')
simple_df.to_pickle('data/simple.pkl')